In [ ]:
import sys, os
from pyprojroot import here

# spyder up to find the root
root = here(project_files=[".root"])


# append to path
sys.path.append(str(root))

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)

import hvplot.xarray
import hvplot.pandas

from inr4ssh._src.operators.finite_diff import calculate_gradient, calculate_laplacian
from inr4ssh._src.preprocess.subset import temporal_subset, spatial_subset
from inr4ssh._src.preprocess.coords import (
    correct_coordinate_labels,
    correct_longitude_domain,
)
from inr4ssh._src.data.ssh_obs import load_ssh_altimetry_data_train

from inr4ssh._src.preprocess.coords import correct_coordinate_labels
from inr4ssh._src.viz.movie import create_movie
from inr4ssh._src.metrics.psd import psd_isotropic
from inr4ssh._src.viz.psd.isotropic import plot_psd_isotropic
from inr4ssh._src.viz.obs import plot_obs_demo

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Data

### Evaluation Field

In [ ]:
url = "/Users/eman/code_projects/data/osse_2022b/dc_qg_eval/dc_qg_eval_*.nc"
url = "/Volumes/EMANS_HDD/data/dc22b_osse/raw/dc_qg_eval/dc_qg_eval_*.nc"
ds_field = xr.open_mfdataset(url)

ds_field = ds_field.rename({"nav_lon": "lon"}).rename({"nav_lat": "lat"})

ds_field = correct_coordinate_labels(ds_field)

ds_field

In [ ]:
ds_field.isel(time=0).ssh.plot.imshow()

In [ ]:
# create_movie(ds_field.ssh_lap, "ssh_field_lap", framedim="time", cmap="RdBu_r")

#### Gradients/Laplacian

In [ ]:
# from inr4ssh._src.operators.finite_diff import calculate_gradient, calculate_laplacian

# ds_field["ssh_grad"] = calculate_gradient(ds_field["ssh"], "longitude", "latitude")
# ds_field["ssh_lap"] = calculate_gradient(ds_field["ssh_grad"], "longitude", "latitude")

# create_movie(ds_field.ssh_grad, "ssh_field_grad", framedim="time", cmap="Spectral_r")
# create_movie(ds_field.ssh_lap, "ssh_field_lap", framedim="time", cmap="RdBu_r")

### PSD

In [ ]:
ds_field_psd = correct_coordinate_labels(ds_field)

# grab ssh
ds_field_psd = ds_field_psd.ssh

# correct units, degrees -> meters
ds_field_psd["longitude"] = ds_field_psd.longitude * 111e3
ds_field_psd["latitude"] = ds_field_psd.latitude * 111e3

# calculate
ds_field_psd = psd_isotropic(ds_field_psd)

In [ ]:
fig, ax = plot_psd_isotropic(ds_field_psd.freq_r.values * 1e3, ds_field_psd.values)
plt.tight_layout()
plt.show()

### PSD - Spatial-Temporal

In [ ]:
from inr4ssh._src.metrics.psd import psd_spacetime, psd_spacetime_dask
from inr4ssh._src.viz.psd.spacetime import (
    plot_psd_spacetime_wavelength,
    plot_psd_spacetime_wavenumber,
)

ds_field_psd = correct_coordinate_labels(ds_field)

# grab ssh
ds_field_psd = ds_field_psd.ssh

# grab ssh
ds_field_psd = ds_field_psd.compute()

# correct units, degrees -> meters
ds_field_psd["longitude"] = ds_field_psd.longitude * 111e3
ds_field_psd["latitude"] = ds_field_psd.latitude * 111e3

time_norm = np.timedelta64(1, "D")
# mean psd of signal
ds_field_psd["time"] = (ds_field_psd.time - ds_field_psd.time[0]) / time_norm

# calculate
ds_field_psd = psd_spacetime_dask(ds_field_psd)

In [ ]:
fig, ax, cbar = plot_psd_spacetime_wavelength(
    ds_field_psd.freq_longitude * 1e3,
    ds_field_psd.freq_time,
    ds_field_psd,
)

plt.tight_layout()
plt.show()

In [ ]:
# fig, ax, cbar = plot_psd_spacetime_wavenumber(
#     ds_field_psd.freq_longitude * 1e3,
#     ds_field_psd.freq_time,
#     ds_field_psd,
# )

# plt.tight_layout()
# plt.show()

## Observations

In [ ]:
# # grab ssh
# ds_field_psd = ds_field.ssh_grad

# # correct units, degrees -> meters
# ds_field_psd["longitude"] = ds_field_psd.longitude * 111e3
# ds_field_psd["latitude"] = ds_field_psd.latitude * 111e3

# # calculate
# ds_field_psd = psd_isotropic(ds_field_psd)

# fig, ax = plot_isotropic_psd(ds_field_psd, freq_scale=1e3)
# ax.set_ylabel(r"PSD [m$^2$s$^{-2}$/cyles/m")
# plt.tight_layout()
# plt.show()

### Missing Time

In [ ]:
ds_obs = xr.open_dataset(
    "/Users/eman/code_projects/data/osse_2022b/dc_qg_obs_fullfields/ssh_obs_fullfields.nc"
)


ds_obs = correct_coordinate_labels(ds_obs)

ds_obs = ds_obs.rename({"ssh": "ssh_obs"})

ds_obs

In [ ]:
ds_obs = xr.merge([ds_field, ds_obs])

#### Movie (GIF)

In [ ]:
# create_movie(ds_obs.ssh_obs, "ssh_missing_time", framedim="time", cmap="viridis")

In [ ]:
# !ls /Users/eman/code_projects/data/osse_2022b/dc_qg_obs_nadirlike/

### Jason-Like

In [ ]:
ds_obs = xr.open_dataset(
    "/Users/eman/code_projects/data/osse_2022b/dc_qg_obs_jasonlike/ssh_obs_jasonlike.nc"
)

ds_obs = ds_obs.sortby("time")

ds_obs = correct_coordinate_labels(ds_obs)

# ds_obs = ds_obs.rename({"ssh": "ssh_obs"})
ds_obs

In [ ]:
central_date = np.datetime64("2012-10-22")
num_days = 1
delta_t = np.timedelta64(num_days, "D")
variable = "ssh_obs"

plot_obs_demo(ds_obs, central_date, delta_t, variable, verbose=True)

#### Gridded Dataset

In [ ]:
from inr4ssh._src.preprocess.obs import bin_observations

In [ ]:
ds_obs_binned = bin_observations(ds_obs, ds_field, "ssh_obs", np.timedelta64(12, "h"))

In [ ]:
ds_obs_binned.isel(time=10).ssh_obs.plot()

#### Movie (GIF)

In [ ]:
# create_movie(ds_obs_binned.ssh_obs, "ssh_jasonlike", framedim="time", cmap="viridis")

### NADIR-Like

In [ ]:
ds_obs = xr.open_mfdataset(
    "/Users/eman/code_projects/data/osse_2022b/dc_qg_obs_nadirlike/ssh_obs*.nc",
    combine="nested",
    concat_dim="time",
    parallel=True,
    preprocess=None,
    engine="netcdf4",
)


ds_obs = ds_obs.sortby("time")

ds_obs = correct_coordinate_labels(ds_obs)

# ds_obs = ds_obs.rename({"ssh": "ssh_obs"})
ds_obs

#### Demo

In [ ]:
central_date = np.datetime64("2012-10-22")
num_days = 10
delta_t = np.timedelta64(num_days, "D")
variable = "ssh_obs"

plot_obs_demo(ds_obs, central_date, delta_t, variable, verbose=True)

#### Gridded Dataset

In [ ]:
ds_obs_binned = bin_observations(ds_obs, ds_field, "ssh_obs", np.timedelta64(12, "h"))

In [ ]:
ds_obs_binned.isel(time=10).ssh_obs.plot()

#### Movie (GIF)

In [ ]:
# create_movie(ds_obs_binned.ssh_obs, "ssh_nadirlike", framedim="time", cmap="viridis")